# Mini project
### Tasks:

- Etap 1: Wczytaj dane, sprawdź ich strukturę i wykonaj wstępną eksplorację.
- Etap 2: filtruj dane sprzedażowe wg wartości i regionu
- Etap 3: policz metryki (np. średni zysk, liczba transakcji) dla wybranych kolumn
- Etap 4: oczyść dane klienta – brakujące dane, duplikaty, błędy
- Etap 5: scal kilka źródeł w jedną tabelę (np. zamówienia + klienci)

Wszystkie kroki, bedą realizowane na poniższym datasetcie.

In [235]:
# Dataset

import pandas as pd
import numpy as np

# Dane sprzedażowe
df_sales = pd.DataFrame({
    'ID': list(range(101, 121)) + [115],  # dodany duplikat ID 115
    'Sprzedawca': [
        'Anna', 'Bartek', 'Anna', 'Kasia', 'Marek', 'Bartek', 'Kasia', 'Anna', 'Marek', 'Bartek',
        'Kasia', 'Anna', 'Bartek', 'Kasia', 'Marek', 'Anna', 'Bartek', 'Kasia', 'Marek', None,  # jeden brak Sprzedawcy
        'Anna'
    ],
    'Produkt': [
        'Laptop', 'Tablet', 'Smartfon', 'Laptop', 'Tablet', 'Laptop', 'Smartfon', 'Laptop', 'Tablet', 'Smartfon',
        'Laptop', None, 'Tablet', 'Smartfon', 'Laptop', 'Tablet', 'Laptop', 'Smartfon', 'Tablet', 'Laptop',
        'Laptop'  # duplikat rekordu 115
    ],
    'Region': [
        'Północ', 'Południe', 'Północ', 'Zachód', 'Wschód', 'Południe', 'Zachód', 'Północ', 'Wschód', None,  # jeden brak Regionu
        'Zachód', 'Północ', 'Południe', 'Zachód', 'Wschód', 'Północ', 'Południe', 'Zachód', 'Wschód', 'Północ',
        'Północ'
    ],
    'Sprzedaż': [
        3000, 1500, 2200, 2700, 1800, 3100, 2000, 2800, np.nan, 1600,  # jedna wartość Sprzedaży to NaN
        2600, 2400, 1550, 2300, 2900, 1950, 3200, 2100, 1750, 3300,
        1950
    ],
    'Klient_ID': [
        1, 2, 1, 3, 4, 2, 5, 6, 7, 8,
        3, 1, 9, 3, 10, 11, 12, 13, 14, 15,
        11
    ]
})

# Dane klientów
df_clients = pd.DataFrame({
    'Klient_ID': list(range(1, 16)) + [16],
    'Nazwa': [
        'Firma A', 'Firma B', 'Firma C', 'Firma D', 'Firma E',
        'Firma F', 'Firma G', 'Firma H', 'Firma I', 'Firma J',
        'Firma K', 'Firma L', 'Firma M', 'Firma N', None,  # brak Nazwa dla ID 15
        'Firma P'
    ],
    'Miasto': [
        'Warszawa', 'Kraków', 'Gdańsk', 'Poznań', 'Łódź',
        'Wrocław', 'Katowice', None, 'Bydgoszcz', 'Lublin',  # brak Miasto dla ID 8
        'Rzeszów', 'Białystok', 'Toruń', 'Zielona Góra', 'Opole',
        'Warszawa'
    ],
    'Branża': [
        'IT', 'Finanse', 'Edukacja', 'Budownictwo', 'Transport',
        'IT', 'Handel', 'Finanse', None, 'IT',  # brak Branża dla ID 9
        'Budownictwo', 'Edukacja', 'Transport', 'IT', 'Handel',
        'IT'
    ]
})

---

### Data exploration

In [236]:
df_sales

,ID,Sprzedawca,Produkt,Region,Sprzedaż,Klient_ID
0,101,Anna,Laptop,Północ,3000.0,1
1,102,Bartek,Tablet,Południe,1500.0,2
2,103,Anna,Smartfon,Północ,2200.0,1
3,104,Kasia,Laptop,Zachód,2700.0,3
4,105,Marek,Tablet,Wschód,1800.0,4
5,106,Bartek,Laptop,Południe,3100.0,2
6,107,Kasia,Smartfon,Zachód,2000.0,5
7,108,Anna,Laptop,Północ,2800.0,6
8,109,Marek,Tablet,Wschód,NaN,7
9,110,Bartek,Smartfon,None,1600.0,8


In [237]:
df_clients

,Klient_ID,Nazwa,Miasto,Branża
0,1,Firma A,Warszawa,IT
1,2,Firma B,Kraków,Finanse
2,3,Firma C,Gdańsk,Edukacja
3,4,Firma D,Poznań,Budownictwo
4,5,Firma E,Łódź,Transport
5,6,Firma F,Wrocław,IT
6,7,Firma G,Katowice,Handel
7,8,Firma H,None,Finanse
8,9,Firma I,Bydgoszcz,None
9,10,Firma J,Lublin,IT


In [238]:
df_sales.info(verbose=True,
              memory_usage=False,
              show_counts=True)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ID          21 non-null     int64  
 1   Sprzedawca  20 non-null     object 
 2   Produkt     20 non-null     object 
 3   Region      20 non-null     object 
 4   Sprzedaż    20 non-null     float64
 5   Klient_ID   21 non-null     int64  
dtypes: float64(1), int64(2), object(3)

In [239]:
df_clients.info(verbose=True,
                memory_usage=False,
                show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Klient_ID  16 non-null     int64 
 1   Nazwa      15 non-null     object
 2   Miasto     15 non-null     object
 3   Branża     15 non-null     object
dtypes: int64(1), object(3)

In [240]:
df_sprzedaz = df_sales['Sprzedaż'].describe(percentiles=[0.25, 0.5, 0.75])
df_sprzedaz

count      20.00000
mean     2335.00000
std       581.08338
min      1500.00000
25%      1912.50000
50%      2250.00000
75%      2825.00000
max      3300.00000
Name: Sprzedaż, dtype: float64

In [241]:
# Usseles for that dataframe
# df_clients.describe()

In [242]:
unique_products = df_sales["Produkt"].unique()
unique_region = df_sales['Region'].unique()
unique_industry = df_clients['Branża'].unique()
unique_seller = df_sales['Sprzedawca'].unique()

print(f"""There are:
    - {len(unique_products)} unique products: {list(unique_products)}
    - {len(unique_region)} unique regions: {list(unique_region)}
    - {len(unique_seller)} unique sellers: {list(unique_seller)}
    - {len(unique_industry)} unique industries: {list(unique_industry)}
""")

There are:
    - 4 unique products: ['Laptop', 'Tablet', 'Smartfon', None]
    - 5 unique regions: ['Północ', 'Południe', 'Zachód', 'Wschód', None]
    - 5 unique sellers: ['Anna', 'Bartek', 'Kasia', 'Marek', None]
    - 7 unique industries: ['IT', 'Finanse', 'Edukacja', 'Budownictwo', 'Transport', 'Handel', None]



In [243]:
# Check if there are any Null values
print(f"""In the dataframe \"df_sales\" {f"there {"is" if df_sales.isna().sum().sum() == 1 else "are"} {df_sales.isna().sum().sum()}" if df_sales.isna().values.any() else "there are not"} null value.
In the dataframe \"df_clients\" {f"there {"is" if df_clients.isna().sum().sum() == 1 else "are"} {df_clients.isna().sum().sum()}" if df_clients.isna().values.any() else "there are not"} null value."
      """)



In the dataframe "df_sales" there are 4 null value.
In the dataframe "df_clients" there are 3 null value."
      


In [244]:
# Best sellers, and their amout of sales
df_sales_grouped = df_sales.groupby(by=['Sprzedawca'], sort=True)
count_sells_by_seller = df_sales_grouped['Sprzedaż'].count()
amout_sells_by_seller = df_sales_grouped['Sprzedaż'].sum()


best_sellers_names = list((count_sells_by_seller[count_sells_by_seller >= count_sells_by_seller.quantile(0.95)]).keys())
best_sellers_amount = amout_sells_by_seller[best_sellers_names].to_dict()


In [245]:
# Sprzedaz na produkt
sum_sales_per_product = df_sales.groupby("Produkt")['Sprzedaż'].sum()
count_sales_per_product = df_sales.groupby("Produkt")['Sprzedaż'].count()
best_selling_product = count_sales_per_product.index[0]

best_selling_product 
best_selling_product_amout = sum_sales_per_product[best_selling_product]


In [246]:
df_merged = pd.merge(df_clients, df_sales)
df_merged

,Klient_ID,Nazwa,Miasto,Branża,ID,Sprzedawca,Produkt,Region,Sprzedaż
0,1,Firma A,Warszawa,IT,101,Anna,Laptop,Północ,3000.0
1,1,Firma A,Warszawa,IT,103,Anna,Smartfon,Północ,2200.0
2,1,Firma A,Warszawa,IT,112,Anna,None,Północ,2400.0
3,2,Firma B,Kraków,Finanse,102,Bartek,Tablet,Południe,1500.0
4,2,Firma B,Kraków,Finanse,106,Bartek,Laptop,Południe,3100.0
5,3,Firma C,Gdańsk,Edukacja,104,Kasia,Laptop,Zachód,2700.0
6,3,Firma C,Gdańsk,Edukacja,111,Kasia,Laptop,Zachód,2600.0
7,3,Firma C,Gdańsk,Edukacja,114,Kasia,Smartfon,Zachód,2300.0
8,4,Firma D,Poznań,Budownictwo,105,Marek,Tablet,Wschód,1800.0
9,5,Firma E,Łódź,Transport,107,Kasia,Smartfon,Zachód,2000.0


In [247]:
df_merged.groupby(by='Region')['Sprzedaż'].sum()

Region
Południe     9350.0
Północ      17600.0
Wschód       6450.0
Zachód      11700.0
Name: Sprzedaż, dtype: float64

In [248]:
df_merged.loc[df_merged['Nazwa'] == df_merged.groupby(by='Nazwa')['Sprzedaż'].sum().index[0]]['Produkt']

0      Laptop
1    Smartfon
2        None
Name: Produkt, dtype: object

In [249]:
d1 = {name: str(dftype) for name, dftype in zip(df_sales.columns, df_sales.dtypes)}
d2 = {name: str(dftype) for name, dftype in zip(df_clients.columns, df_clients.dtypes)}
print(f"""Conclusion:
      We've got two datasets: ['df_sales', 'df_clients']
      
      The first dataset consists {len(df_sales.columns)} columns: {d1},
      and in the \"df_sales\" {f"there {"is" if df_sales.isna().sum().sum() == 1 else "are"} {df_sales.isna().sum().sum()}" if df_sales.isna().values.any() else "there are 0"} null value.
      About data:
      Mean sale is equal to {df_sprzedaz.__getitem__('mean')}.
      Median sale was equal to {df_sprzedaz.__getitem__('50%')}
      Minimum sale was equal to {df_sprzedaz.__getitem__('min')} and the maximum was {df_sprzedaz.__getitem__('max')}.
      We've got {len(df_sales['Sprzedawca'].unique())} sellers: {list(df_sales['Sprzedawca'].unique())} who took part in {len(df_sales['Sprzedawca'])} sales.
      We can observe {len(df_sales['Produkt'].unique())} unique product, which are: {list(df_sales['Produkt'].unique())}
      Also Sales are distributed between {len(df_sales['Region'].unique())} regions, like {list(df_sales['Region'].unique())}

      The second dataset consists {len(df_clients.columns)} columns: {d2}
      and in the \"df_clients\" {f"there {"is" if df_clients.isna().sum().sum() == 1 else "are"} {df_clients.isna().sum().sum()}" if df_clients.isna().values.any() else "there are 0"} null value.
      About data:
      We've got {len(df_clients['Nazwa'].unique())} clients from {len(df_clients['Branża'].unique())} industries: {list(df_clients['Branża'].unique())}, which operates on different cities: {list(df_clients['Miasto'])}.
      
      Additional info:
      We can highlight {len(best_sellers_names)} best sellers, which are {best_sellers_names}. They've sold sum of {best_sellers_amount}.
      Also worth mentioning is product review, the best selling product is {best_selling_product} with sum of sales equal to {best_selling_product_amout}.
      About the firms, we can highlight the {df_merged.groupby(by='Nazwa')['Sprzedaż'].sum().index[0]} which bought {df_merged.loc[df_merged['Nazwa'] == df_merged.groupby(by='Nazwa')['Sprzedaż'].sum().index[0]]['Produkt'].values} for the value of {df_merged.groupby(by='Nazwa')['Sprzedaż'].sum().iloc[0]}.
      """)

Conclusion:
      We've got two datasets: ['df_sales', 'df_clients']
      
      The first dataset consists 6 columns: {'ID': 'int64', 'Sprzedawca': 'object', 'Produkt': 'object', 'Region': 'object', 'Sprzedaż': 'float64', 'Klient_ID': 'int64'},
      and in the "df_sales" there are 4 null value.
      About data:
      Mean sale is equal to 2335.0.
      Median sale was equal to 2250.0
      Minimum sale was equal to 1500.0 and the maximum was 3300.0.
      We've got 5 sellers: ['Anna', 'Bartek', 'Kasia', 'Marek', None] who took part in 21 sales.
      We can observe 4 unique product, which are: ['Laptop', 'Tablet', 'Smartfon', None]
      Also Sales are distributed between 5 regions, like ['Północ', 'Południe', 'Zachód', 'Wschód', None]

      The second dataset consists 4 columns: {'Klient_ID': 'int64', 'Nazwa': 'object', 'Miasto': 'object', 'Branża': 'object'}
      and in the "df_clients" there are 3 null value.
      About data:
      We've got 16 clients from 7 industries: ['IT

---

### Data filtering


In [250]:
# Sales over 2500
df_sales.loc[df_sales['Sprzedaż'] >= 2500] # Sales over 2500

,ID,Sprzedawca,Produkt,Region,Sprzedaż,Klient_ID
0,101,Anna,Laptop,Północ,3000.0,1
3,104,Kasia,Laptop,Zachód,2700.0,3
5,106,Bartek,Laptop,Południe,3100.0,2
7,108,Anna,Laptop,Północ,2800.0,6
10,111,Kasia,Laptop,Zachód,2600.0,3
14,115,Marek,Laptop,Wschód,2900.0,10
16,117,Bartek,Laptop,Południe,3200.0,12
19,120,None,Laptop,Północ,3300.0,15


In [251]:
df_sales.loc[df_sales['Region'] == 'Północ'] # Sales in Region North

,ID,Sprzedawca,Produkt,Region,Sprzedaż,Klient_ID
0,101,Anna,Laptop,Północ,3000.0,1
2,103,Anna,Smartfon,Północ,2200.0,1
7,108,Anna,Laptop,Północ,2800.0,6
11,112,Anna,None,Północ,2400.0,1
15,116,Anna,Tablet,Północ,1950.0,11
19,120,None,Laptop,Północ,3300.0,15
20,115,Anna,Laptop,Północ,1950.0,11


In [252]:
df_sales.loc[(df_sales['Region'].isin(['Północ', 'Zachód'])) & (df_sales['Produkt'] == 'Laptop')] # Sales in two regions where laptops were sold

,ID,Sprzedawca,Produkt,Region,Sprzedaż,Klient_ID
0,101,Anna,Laptop,Północ,3000.0,1
3,104,Kasia,Laptop,Zachód,2700.0,3
7,108,Anna,Laptop,Północ,2800.0,6
10,111,Kasia,Laptop,Zachód,2600.0,3
19,120,None,Laptop,Północ,3300.0,15
20,115,Anna,Laptop,Północ,1950.0,11


In [253]:
df_sales.loc[df_sales['Sprzedaż'].between(2000, 3000)] # Sales ranging from 2000 to 3000

,ID,Sprzedawca,Produkt,Region,Sprzedaż,Klient_ID
0,101,Anna,Laptop,Północ,3000.0,1
2,103,Anna,Smartfon,Północ,2200.0,1
3,104,Kasia,Laptop,Zachód,2700.0,3
6,107,Kasia,Smartfon,Zachód,2000.0,5
7,108,Anna,Laptop,Północ,2800.0,6
10,111,Kasia,Laptop,Zachód,2600.0,3
11,112,Anna,None,Północ,2400.0,1
13,114,Kasia,Smartfon,Zachód,2300.0,3
14,115,Marek,Laptop,Wschód,2900.0,10
17,118,Kasia,Smartfon,Zachód,2100.0,13


In [254]:
df_sales.loc[(df_sales['Region'] == 'Południe') & (df_sales['Produkt'] == "Smartfon")]['Sprzedawca']

Series([], Name: Sprzedawca, dtype: object)

In [255]:
df_sales.loc[(~ (df_sales['Produkt'] == 'Laptop')) & (df_sales['Sprzedaż'] < 2000)]

,ID,Sprzedawca,Produkt,Region,Sprzedaż,Klient_ID
1,102,Bartek,Tablet,Południe,1500.0,2
4,105,Marek,Tablet,Wschód,1800.0,4
9,110,Bartek,Smartfon,None,1600.0,8
12,113,Bartek,Tablet,Południe,1550.0,9
15,116,Anna,Tablet,Północ,1950.0,11
18,119,Marek,Tablet,Wschód,1750.0,14


In [256]:
df_merged

,Klient_ID,Nazwa,Miasto,Branża,ID,Sprzedawca,Produkt,Region,Sprzedaż
0,1,Firma A,Warszawa,IT,101,Anna,Laptop,Północ,3000.0
1,1,Firma A,Warszawa,IT,103,Anna,Smartfon,Północ,2200.0
2,1,Firma A,Warszawa,IT,112,Anna,None,Północ,2400.0
3,2,Firma B,Kraków,Finanse,102,Bartek,Tablet,Południe,1500.0
4,2,Firma B,Kraków,Finanse,106,Bartek,Laptop,Południe,3100.0
5,3,Firma C,Gdańsk,Edukacja,104,Kasia,Laptop,Zachód,2700.0
6,3,Firma C,Gdańsk,Edukacja,111,Kasia,Laptop,Zachód,2600.0
7,3,Firma C,Gdańsk,Edukacja,114,Kasia,Smartfon,Zachód,2300.0
8,4,Firma D,Poznań,Budownictwo,105,Marek,Tablet,Wschód,1800.0
9,5,Firma E,Łódź,Transport,107,Kasia,Smartfon,Zachód,2000.0


In [257]:
df_merged.loc[(df_merged['Miasto'] == 'Poznań') & (~ (df_merged['Produkt'] == 'Smarfon'))] 

,Klient_ID,Nazwa,Miasto,Branża,ID,Sprzedawca,Produkt,Region,Sprzedaż
8,4,Firma D,Poznań,Budownictwo,105,Marek,Tablet,Wschód,1800.0


In [258]:
df_merged.loc[(df_merged['Miasto'] == 'Warszawa') & (df_merged['Produkt'] == 'Laptop') & (df_merged['Sprzedaż'] > df_merged['Sprzedaż'].mean())]

,Klient_ID,Nazwa,Miasto,Branża,ID,Sprzedawca,Produkt,Region,Sprzedaż
0,1,Firma A,Warszawa,IT,101,Anna,Laptop,Północ,3000.0


In [259]:
df_merged.loc[(df_merged['Branża'] == 'IT') & (df_merged['Sprzedaż'] > 2000)]

,Klient_ID,Nazwa,Miasto,Branża,ID,Sprzedawca,Produkt,Region,Sprzedaż
0,1,Firma A,Warszawa,IT,101,Anna,Laptop,Północ,3000.0
1,1,Firma A,Warszawa,IT,103,Anna,Smartfon,Północ,2200.0
2,1,Firma A,Warszawa,IT,112,Anna,None,Północ,2400.0
10,6,Firma F,Wrocław,IT,108,Anna,Laptop,Północ,2800.0
14,10,Firma J,Lublin,IT,115,Marek,Laptop,Wschód,2900.0


In [260]:
df_merged.loc[(df_merged['Produkt'] == 'Tablet') & (~(df_merged['Branża'] == 'Edukacja')), 'Sprzedawca']

3     Bartek
8      Marek
11     Marek
13    Bartek
15      Anna
19     Marek
Name: Sprzedawca, dtype: object

---

### Cleaning data

In [261]:
df_clients

,Klient_ID,Nazwa,Miasto,Branża
0,1,Firma A,Warszawa,IT
1,2,Firma B,Kraków,Finanse
2,3,Firma C,Gdańsk,Edukacja
3,4,Firma D,Poznań,Budownictwo
4,5,Firma E,Łódź,Transport
5,6,Firma F,Wrocław,IT
6,7,Firma G,Katowice,Handel
7,8,Firma H,None,Finanse
8,9,Firma I,Bydgoszcz,None
9,10,Firma J,Lublin,IT


In [262]:
drop_na = True

if drop_na:
    # We cannot determine the id by the name because everyone have different ID and the name can't be determined by ID.
    # In df_clients we can't logically relate rows of each firm to id, because every firm is different as well as ID, from that we cannot also relate industry, so I decided to drop na fields instead of filling them with random values.
    df_sales.drop_duplicates(subset=['ID'], keep='first', inplace=True)
    df_sales.dropna(subset=['Sprzedawca'],inplace=True)
    df_clients.dropna(subset=['Branża', 'Miasto', 'Nazwa'], inplace=True)
else:
    df_sales.fillna(method='ffill')

In [263]:
df_sales.drop_duplicates(subset=['ID'], keep='first', inplace=True)

In [264]:
if df_sales['Sprzedaż'].isna().sum() == 1:
    df_sales['Sprzedaż'].fillna(df_sales.loc[df_sales['Sprzedawca'] == df_sales[df_sales['Sprzedaż'].isna()].iloc[0,1], 'Sprzedaż'].mean(), inplace=True)
else:
    df_sales['Sprzedaż'].fillna(df_sales['Sprzedaż'].mean(), inplace=True)

/var/folders/4g/5v3gx6nx2ml_1gtsc_n5xnjc0000gn/T/ipykernel_24632/1919078387.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_sales['Sprzedaż'].fillna(df_sales.loc[df_sales['Sprzedawca'] == df_sales[df_sales['Sprzedaż'].isna()].iloc[0,1], 'Sprzedaż'].mean(), inplace=True)


In [265]:
if df_sales['Region'].isna().sum() == 1:
    df_sales['Region'].fillna(df_sales.loc[df_sales['Sprzedawca'] == df_sales[df_sales['Region'].isna()].iloc[0,1], 'Region'].mode()[0], inplace=True)
else:
    df_sales['Region'].fillna(df_sales['Region'].mode()[0], inplace=True)

/var/folders/4g/5v3gx6nx2ml_1gtsc_n5xnjc0000gn/T/ipykernel_24632/2458264317.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_sales['Region'].fillna(df_sales.loc[df_sales['Sprzedawca'] == df_sales[df_sales['Region'].isna()].iloc[0,1], 'Region'].mode()[0], inplace=True)


---

### Metrics

In [267]:
df_merged = pd.merge(df_clients, df_sales)
df_merged

,Klient_ID,Nazwa,Miasto,Branża,ID,Sprzedawca,Produkt,Region,Sprzedaż
0,1,Firma A,Warszawa,IT,101,Anna,Laptop,Północ,3000.0
1,1,Firma A,Warszawa,IT,103,Anna,Smartfon,Północ,2200.0
2,1,Firma A,Warszawa,IT,112,Anna,None,Północ,2400.0
3,2,Firma B,Kraków,Finanse,102,Bartek,Tablet,Południe,1500.0
4,2,Firma B,Kraków,Finanse,106,Bartek,Laptop,Południe,3100.0
5,3,Firma C,Gdańsk,Edukacja,104,Kasia,Laptop,Zachód,2700.0
6,3,Firma C,Gdańsk,Edukacja,111,Kasia,Laptop,Zachód,2600.0
7,3,Firma C,Gdańsk,Edukacja,114,Kasia,Smartfon,Zachód,2300.0
8,4,Firma D,Poznań,Budownictwo,105,Marek,Tablet,Wschód,1800.0
9,5,Firma E,Łódź,Transport,107,Kasia,Smartfon,Zachód,2000.0


In [ ]:
# Sum of sales for every seller
df_merged.groupby(by=['Sprzedawca'])['Sprzedaż'].sum()

Sprzedawca
Anna      12350.0
Bartek     7800.0
Kasia     11700.0
Marek      8600.0
Name: Sprzedaż, dtype: float64

In [271]:
df_merged.groupby(by='Region')['ID'].count() # Number of transactions

Region
Południe    3
Północ      5
Wschód      4
Zachód      5
Name: ID, dtype: int64

In [ ]:
df_merged.groupby(by='Nazwa', sort=True)['Klient_ID'].count().index[0] # company with the most transactions

'Firma A'

In [ ]:
# Mean sales and sum of sales for each industry

df_merged.groupby(by='Branża', as_index=True).agg(Sum = ('Sprzedaż', 'sum'),
                                   Mean = ('Sprzedaż', 'mean'))

,Sum,Mean
Branża,,
Budownictwo,3750.0,1875.000000
Edukacja,10800.0,2700.000000
Finanse,4600.0,2300.000000
Handel,2150.0,2150.000000
IT,15050.0,2508.333333
Transport,4100.0,2050.000000


In [300]:
df_merged.groupby(by=['Sprzedawca'])['Nazwa'].nunique() # Number of companies distributed by each seller

Sprzedawca
Anna      3
Bartek    2
Kasia     3
Marek     4
Name: Nazwa, dtype: int64

In [ ]:
df_merged.groupby(by=['Miasto'])['Sprzedaż'].count() # Number of sales for each city

Miasto
Białystok       1
Gdańsk          3
Katowice        1
Kraków          2
Lublin          1
Poznań          1
Rzeszów         1
Toruń           1
Warszawa        3
Wrocław         1
Zielona Góra    1
Łódź            1
Name: Sprzedaż, dtype: int64

In [316]:
# Minimum and maximum sale for each industry

df_merged.groupby(by='Branża').agg(Min = ('Sprzedaż', 'min'),
                                               Max = ('Sprzedaż', 'max')) 

,Min,Max
Branża,,
Budownictwo,1800.0,1950.0
Edukacja,2300.0,3200.0
Finanse,1500.0,3100.0
Handel,2150.0,2150.0
IT,1750.0,3000.0
Transport,2000.0,2100.0


In [320]:
df_merged.groupby(by='Branża')['Sprzedaż'].sum().index[0] # Industry with the biggest sum of sales

'Budownictwo'

In [321]:
df_merged

,Klient_ID,Nazwa,Miasto,Branża,ID,Sprzedawca,Produkt,Region,Sprzedaż
0,1,Firma A,Warszawa,IT,101,Anna,Laptop,Północ,3000.0
1,1,Firma A,Warszawa,IT,103,Anna,Smartfon,Północ,2200.0
2,1,Firma A,Warszawa,IT,112,Anna,None,Północ,2400.0
3,2,Firma B,Kraków,Finanse,102,Bartek,Tablet,Południe,1500.0
4,2,Firma B,Kraków,Finanse,106,Bartek,Laptop,Południe,3100.0
5,3,Firma C,Gdańsk,Edukacja,104,Kasia,Laptop,Zachód,2700.0
6,3,Firma C,Gdańsk,Edukacja,111,Kasia,Laptop,Zachód,2600.0
7,3,Firma C,Gdańsk,Edukacja,114,Kasia,Smartfon,Zachód,2300.0
8,4,Firma D,Poznań,Budownictwo,105,Marek,Tablet,Wschód,1800.0
9,5,Firma E,Łódź,Transport,107,Kasia,Smartfon,Zachód,2000.0


In [ ]:
df_merged.groupby(by=['Sprzedawca'])['Sprzedaż'].mean().sort_values(ascending=False)

Sprzedawca
Bartek    2600.0
Anna      2470.0
Kasia     2340.0
Marek     2150.0
Name: Sprzedaż, dtype: float64